# Tutorial 01. Block Poisson problem (case with None on the diagonal).

In this tutorial we first solve the problem

\begin{cases}
-u'' = f & \text{in }\Omega = (0, 1),\\
 u   = 0 & \text{on }\Gamma = \{0, 1\}
\end{cases}

using non blocked FEniCSx code.

Then we use block support of FEniCSx to solve the system

\begin{cases}
- w_1'' - 2 w_2'' = 3 f & \text{in }\Omega,\\
- 3 w_1'' = 3 f & \text{in }\Omega
\end{cases}

subject to

\begin{cases}
 w_1 = 0 & \text{on }\Gamma,\\
 w_2 = 0 & \text{on }\Gamma
\end{cases}

By construction the solution of the system is
$$(w_1, w_2) = (u, u)$$

This example is meant to show how it is currently not possible to have `None` block diagonal terms, that would result in a wrong application of the boundary conditions.

In [ ]:
import numpy as np
from mpi4py import MPI
from petsc4py import PETSc
from ufl import dx, grad, inner, sin, SpatialCoordinate, TestFunction, TrialFunction
from dolfinx import Constant, DirichletBC, Function, FunctionSpace, solve, UnitIntervalMesh
from dolfinx.fem import (assemble_matrix_block, assemble_scalar, assemble_vector_block,
                         BlockVecSubVectorWrapper, create_vector_block, locate_dofs_topological)
from dolfinx.mesh import locate_entities_geometrical
from dolfinx.plotting import plot

### Mesh generation

In [ ]:
mesh = UnitIntervalMesh(MPI.COMM_WORLD, 32)

In [ ]:
def left(x):
    return abs(x[0] - 0.) < np.finfo(float).eps


def right(x):
    return abs(x[0] - 1.) < np.finfo(float).eps


left_facets = locate_entities_geometrical(mesh, mesh.topology.dim - 1, left)
right_facets = locate_entities_geometrical(mesh, mesh.topology.dim - 1, right)
boundary_facets = np.hstack((left_facets, right_facets))

In [ ]:
x0 = SpatialCoordinate(mesh)[0]

### Standard case (solve for $u$ with FEniCSx)

In [ ]:
def run_standard():
    # Define a function space
    V = FunctionSpace(mesh, ("Lagrange", 2))
    u = TrialFunction(V)
    v = TestFunction(V)

    # Define problems forms
    a = inner(grad(u), grad(v)) * dx
    f = 100 * sin(20 * x0) * v * dx

    # Define boundary conditions
    zero = Function(V)
    with zero.vector.localForm() as zero_local:
        zero_local.set(0.0)
    bdofs_V = locate_dofs_topological(V, mesh.topology.dim - 1, boundary_facets)
    bc = DirichletBC(zero, bdofs_V)

    # Solve the linear system
    u = Function(V)
    solve(a == f, u, bc,
          petsc_options={"ksp_type": "preonly", "pc_type": "lu", "pc_factor_mat_solver_type": "mumps"})
    u.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

    # Return the solution
    return u

In [ ]:
u = run_standard()

In [ ]:
plot(u)

### Standard block case (solve for ($w_1$, $w_2$) with FEniCSx)

In [ ]:
def run_standard_block(with_diagonal_None):
    # Define a block function space
    V1 = FunctionSpace(mesh, ("Lagrange", 2))
    V2 = FunctionSpace(mesh, ("Lagrange", 2))
    (u1, u2) = (TrialFunction(V1), TrialFunction(V2))
    (v1, v2) = (TestFunction(V1), TestFunction(V2))

    # Define problem block forms
    aa = [[1 * inner(grad(u1), grad(v1)) * dx, 2 * inner(grad(u2), grad(v1)) * dx],
          [3 * inner(grad(u1), grad(v2)) * dx, None]]
    ff = [300 * sin(20 * x0) * v1 * dx,
          300 * sin(20 * x0) * v2 * dx]
    if not with_diagonal_None:
        aa[1][1] = Constant(mesh, 0) * u2 * v2 * dx

    # Define block boundary conditions
    zero = Function(V1)
    with zero.vector.localForm() as zero_local:
        zero_local.set(0.0)
    bdofs_V1 = locate_dofs_topological((V1, V1), mesh.topology.dim - 1, boundary_facets)
    bdofs_V2 = locate_dofs_topological((V2, V1), mesh.topology.dim - 1, boundary_facets)
    bc1 = DirichletBC(zero, bdofs_V1, V1)
    bc2 = DirichletBC(zero, bdofs_V2, V2)
    bcs = [bc1, bc2]

    # Assemble the block linear system
    AA = assemble_matrix_block(aa, bcs)
    AA.assemble()
    FF = assemble_vector_block(ff, aa, bcs)

    # Solve the block linear system
    uu = create_vector_block(ff)
    ksp = PETSc.KSP()
    ksp.create(mesh.mpi_comm())
    ksp.setOperators(AA)
    ksp.setType("preonly")
    ksp.getPC().setType("lu")
    ksp.getPC().setFactorSolverType("mumps")
    ksp.setFromOptions()
    ksp.solve(FF, uu)
    uu.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

    # Split the block solution in components
    u1u2 = (Function(V1), Function(V2))
    with BlockVecSubVectorWrapper(uu, [V1.dofmap, V2.dofmap]) as uu_wrapper:
        for u_wrapper_local, component in zip(uu_wrapper, u1u2):
            with component.vector.localForm() as component_local:
                component_local[:] = u_wrapper_local

    # Return the block solution components
    return u1u2

In [ ]:
u1_false, u2_false = run_standard_block(False)

In [ ]:
plot(u1_false)

In [ ]:
plot(u2_false)

In [ ]:
u1_true, u2_true = run_standard_block(True)

In [ ]:
plot(u1_true)

In [ ]:
plot(u2_true)

### Error computation between the different cases

In [ ]:
def compute_errors(u1, u2, uu1, uu2):
    u_1_norm = np.sqrt(mesh.mpi_comm().allreduce(assemble_scalar(inner(grad(u1), grad(u1)) * dx), op=MPI.SUM))
    u_2_norm = np.sqrt(mesh.mpi_comm().allreduce(assemble_scalar(inner(grad(u2), grad(u2)) * dx), op=MPI.SUM))
    err_1_norm = np.sqrt(
        mesh.mpi_comm().allreduce(assemble_scalar(inner(grad(u1 - uu1), grad(u1 - uu1)) * dx), op=MPI.SUM))
    err_2_norm = np.sqrt(
        mesh.mpi_comm().allreduce(assemble_scalar(inner(grad(u2 - uu2), grad(u2 - uu2)) * dx), op=MPI.SUM))
    print("  Relative error for first component is equal to", err_1_norm / u_1_norm)
    print("  Relative error for second component is equal to", err_2_norm / u_2_norm)
    assert np.isclose(err_1_norm / u_1_norm, 0., atol=1.e-10)
    assert np.isclose(err_2_norm / u_2_norm, 0., atol=1.e-10)

In [ ]:
print("Computing errors between standard and standard block (without None on the diagonal)")
compute_errors(u, u, u1_false, u2_false)

In [ ]:
print("Computing errors between standard and standard block (with None on the diagonal)")
try:
    compute_errors(u, u, u1_true, u2_true)
except AssertionError:
    pass  # Failure is expected
else:
    raise RuntimeError("Assertions were supposed to fail")